In [1]:
%reset -f
import h5py
import time as t
import numpy as np
import scipy as sp
import scipy.io as spi
import tensorflow as tf
import matplotlib.pyplot as plt

data_train = spi.loadmat('train_PDE_ADVD.mat')

u_in = data_train['X_train0']
x_t_in = data_train['X_train1']
s_in = data_train['y_train']

u_in.shape, x_t_in.shape, s_in.shape

((100000, 100), (100000, 2), (100000, 1))

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Dense

bs = 100000

def fn(x):
    y = tf.einsum("ij, ij->i", x[0], x[1])
    y = tf.expand_dims(y, axis = 1)
    return y

hln = 35

inputsB = Input(shape = (100,), name = 'inputsB')
hiddenB = Dense(hln, activation = "relu")(inputsB)
hiddenB = Dense(hln, activation = "relu")(hiddenB)
hiddenB = Dense(hln, activation = "relu")(hiddenB)

inputsT = Input(shape = (2,), name = 'inputsT')
hiddenT = Dense(hln, activation = "relu")(inputsT)
hiddenT = Dense(hln, activation = "relu")(hiddenT)
hiddenT = Dense(hln, activation = "relu")(hiddenT)

combined = Lambda(fn, output_shape = [None, 1])([hiddenB, hiddenT])
output = Dense(1)(combined)

model = Model(inputs = [inputsB, inputsT], outputs = output)    

model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001), loss = 'mse')
model.summary()

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = './ChkPts/CIV_VDN_S1_10000/',
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)

itr = 0
for i in range(0, 40):
    print('Iteration: '+str(itr))
    itr = itr+1
    model.fit({"inputsB":u_in, "inputsT":x_t_in}, s_in, epochs = 1,
              verbose = 1, batch_size = bs, callbacks = [model_checkpoint_callback])    

    itr_ps = 250-1
    model.fit({"inputsB":u_in, "inputsT":x_t_in}, s_in, epochs = itr_ps,
              verbose = 0, batch_size = bs, callbacks = [model_checkpoint_callback])
    itr = itr+itr_ps

print('Total iterations: '+str(itr))

model.load_weights('./ChkPts/CIV_VDN_S1_10000/')

model.save_weights('./model/CIV_VDN_S1_10000')

# model.load_weights('./model/Dense_model_TF_weights_CIV_VDN')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputsB (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 inputsT (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 35)           3535        ['inputsB[0][0]']                
                                                                                                  
 dense_3 (Dense)                (None, 35)           105         ['inputsT[0][0]']                
                                                                                              

In [3]:
data_test = spi.loadmat('test_PDE_ADVD.mat')

u_in_test = data_test['X_test0']
x_t_in_test = data_test['X_test1']
s_in_test = data_test['y_test']

pred = model({"inputsB":u_in_test, "inputsT":x_t_in_test})
    
print()
print(np.mean((s_in_test-pred)**2))
print(np.mean((s_in_test)**2))
print(np.mean((s_in_test-pred)**2)/np.mean((s_in_test)**2))    


0.0024013603
0.41757784508986057
0.005750688976560404
